In [ ]:
import pandas as pd
import geopandas as gpd
import unicodedata,time,os,csv
from datetime import datetime
import requests
import os, glob
import io
from zipfile import ZipFile

Téléchargement des aires urbaines

In [ ]:
url =  r"https://www.insee.fr/fr/statistiques/fichier/4803954/AAV2020_au_01-01-2022.zip"

r = requests.get(url, allow_redirects=True)
open('AAV2020_au_01-01-2022.zip', 'wb').write(r.content)

def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn:
        return zf.read(extract_fn)
    else:
        return {name:zf.read(name) for name in zf.namelist()}
    
df_ZAU = pd.read_excel(io.BytesIO(read_zip(r'AAV2020_au_01-01-2022.zip', 'AAV2020_au_01-01-2022.xlsx')),sheet_name=1,skiprows=5)

df_ZAU.head()

Récupération dans  de la couche des communes

In [ ]:
# url =  r"https://www.insee.fr/fr/statistiques/fichier/4803954/fonds_aav2020_2022.zip"
df_com = gpd.GeoDataFrame.from_file("./data/communes_2022/COMMUNE.shp")
df_com.rename(columns={'INSEE_COM':'CODGEO'},inplace=True)

In [ ]:
df_com = pd.merge(df_com,df_ZAU[["CODGEO","CATEAAV2020",'AAV2020']],left_on='CODGEO', right_on='CODGEO')

In [ ]:
#groupby des ZAU avec somme de la population
df_ZAUgpy = df_com.groupby(['AAV2020'])['POPULATION'].sum().reset_index()

# Jointure du groupby avec les communes
df_com = pd.merge(df_com,df_ZAUgpy,left_on='AAV2020', right_on='AAV2020')

# On renomme les colonnes
df_com.rename(columns={"POPULATION_x": "population", "POPULATION_y": "poppuzau"},inplace=True)

# On supprime les colonnes inutiles
df_com.drop(columns=["INSEE_CAN",'INSEE_ARR','ID','SIREN_EPCI'],inplace= True)

In [ ]:
df_com = df_com.to_crs("EPSG:3857")

In [ ]:
df_com.to_file("com_zau.gpkg", layer='com_zau', driver="GPKG")